In [1]:
import pandas as pd
import numpy as np

import neologdn
import MeCab

import re

In [2]:
text='「第一種過誤」を「αエラー」ともいうのですが、「α」をあわてんぼうの「あ」とかけて、あわてんぼうで、本当はない、2集団の差異をあるように判定してしまう。という論理です。一方「第二種過誤」は「βエラー」ともいいますが、「ベーター」をぼんやりとかけて、ぼんやりのβとしています。こちらも、ぼんやりしていて、本当はあるはずの貴重な2集団の差異！を見逃してしまう様を表現しています。'
# 引用URL：http://www.kokushi.space/?p=1165

### neologdnで文章の正規化

In [3]:
text_normalization = neologdn.normalize(text)

### Mecab + neologd 辞書による形態素解析

In [4]:
neologd_tagger = MeCab.Tagger('-Ochasen -d C:\mecab-ipadic-neologd')

# neologd_tagger.parse(text)で各単語の原形、品詞などが1行で連続して表示される
# 原形、品詞などの間には「\t」が、分かち書きされた単語と単語の区切りには「\n」が表示される
# まずはparseで分かち書きした単語群は1つの文字列型になっているので「\n」で区切り、リスト型にする
wakati_text = neologd_tagger.parse(text_normalization).split('\n')

# 「\n」で区切り、リスト型にした結果の例は下記
# ['空い\tアイ\t空く\t動詞-自立\t五段・カ行イ音便\t連用タ接続', ：リスト0番目
# 'た\tタ\tた\t助動詞\t特殊・タ\t基本形',：リスト1番目
# '時間\tジカン\t時間\t名詞-副詞可能\t\t',：リスト2番目

## 形態素解析結果を格納したリストから特定の品詞を抽出
2つのパターンを記述

### 1.形態素解析結果を格納したリストから特定の品詞（品詞詳細部分まで考慮に入れた場合）のみ抽出

- 「単語、品詞、原形、」などの一連の出力結果を格納したリスト：wakati_list
- 「原形」のみ格納したリスト：original_form_list

これら2つのリストを作成

In [5]:
# 抽出したい品詞のリストを作成（完全一致）
# 品詞参考URL：http://miner.hatenablog.com/entry/323
hinshi_list = ['名詞-一般', '名詞-形容動詞語幹', '名詞-固有名詞-一般',  '名詞-サ変接続', '形容詞-自立', '形容詞-接尾', '形容詞-非自立', '動詞-自立', '動詞-接尾', '動詞-非自立', '副詞-一般', '副詞-助詞類接続']
# hinshi_list = ('名詞-一般', '名詞-サ変接続', '名詞-固有名詞', '名詞-形容動詞語幹'...)とタプルでも同じ結果

wakati_list = []# 「単語、品詞、原形、」などの一連の出力結果を格納
original_form_list =[]# 「原形」のみ格納

# parse() の出力結果の最後は「EOS」という文字のみ
# EOSのとき、pos = wakati.split('\t')[3]の要素はないので下記forループを実行すると「list index out of range」とエラーを発生させてしまう
# よってEOSのときは条件分岐if~breakでforループから抜け出すよう記述
for wakati in wakati_text:
    surface = wakati.split('\t')[0]
    if surface == 'EOS':
        break
    else:
        pos = wakati.split('\t')[3]
        if pos in hinshi_list:# posはhinshi_listの中の要素と完全一致していないと抽出できない
            wakati_list.append(wakati)
            original_form_list.append(wakati.split('\t')[2])# 「原形」はインデックス2番目の要素にあるので

In [6]:
wakati_list

['第一種過誤\tダイイッシュカゴ\t第一種過誤\t名詞-固有名詞-一般\t\t',
 'エラー\tエラー\tエラー\t名詞-サ変接続\t\t',
 'いう\tイウ\tいう\t動詞-自立\t五段・ワ行促音便\t基本形',
 'あわ\tアワ\tあわ\t名詞-一般\t\t',
 'てんぼう\tテンボウ\tてんぼう\t名詞-サ変接続\t\t',
 'かけ\tカケ\tかける\t動詞-自立\t一段\t連用形',
 'てんぼう\tテンボウ\tてんぼう\t名詞-サ変接続\t\t',
 '本当は\tホントウハ\t本当は\t副詞-一般\t\t',
 'ない\tナイ\tない\t形容詞-自立\t形容詞・アウオ段\t基本形',
 '集団\tシュウダン\t集団\t名詞-一般\t\t',
 '差異\tサイ\t差異\t名詞-一般\t\t',
 'ある\tアル\tある\t動詞-自立\t五段・ラ行\t基本形',
 '判定\tハンテイ\t判定\t名詞-サ変接続\t\t',
 'し\tシ\tする\t動詞-自立\tサ変・スル\t連用形',
 'しまう\tシマウ\tしまう\t動詞-非自立\t五段・ワ行促音便\t基本形',
 '論理\tロンリ\t論理\t名詞-一般\t\t',
 '第二種\tダイニシュ\t第二種\t名詞-固有名詞-一般\t\t',
 '過誤\tカゴ\t過誤\t名詞-一般\t\t',
 'エラー\tエラー\tエラー\t名詞-サ変接続\t\t',
 'いい\tイイ\tいう\t動詞-自立\t五段・ワ行促音便\t連用形',
 'ベーター\tベーター\tベーター\t名詞-固有名詞-一般\t\t',
 'ぼんやり\tボンヤリ\tぼんやり\t副詞-一般\t\t',
 'かけ\tカケ\tかける\t動詞-自立\t一段\t連用形',
 'ぼんやり\tボンヤリ\tぼんやり\t名詞-サ変接続\t\t',
 'し\tシ\tする\t動詞-自立\tサ変・スル\t連用形',
 'い\tイ\tいる\t動詞-非自立\t一段\t連用形',
 'ぼんやり\tボンヤリ\tぼんやり\t名詞-サ変接続\t\t',
 'し\tシ\tする\t動詞-自立\tサ変・スル\t連用形',
 'い\tイ\tいる\t動詞-非自立\t一段\t連用形',
 '本当は\tホントウハ\t本当は\t副詞-一般\t\t',
 'ある\tアル\tある\t動詞-自立\t

In [7]:
original_form_list

['第一種過誤',
 'エラー',
 'いう',
 'あわ',
 'てんぼう',
 'かける',
 'てんぼう',
 '本当は',
 'ない',
 '集団',
 '差異',
 'ある',
 '判定',
 'する',
 'しまう',
 '論理',
 '第二種',
 '過誤',
 'エラー',
 'いう',
 'ベーター',
 'ぼんやり',
 'かける',
 'ぼんやり',
 'する',
 'いる',
 'ぼんやり',
 'する',
 'いる',
 '本当は',
 'ある',
 '貴重',
 '集団',
 '差異',
 '見逃す',
 'しまう',
 '表現',
 'する',
 'いる']

### 2.形態素解析結果を格納したリストから特定の品詞（純粋な品詞名）のみ抽出
- 「単語、品詞、原形、」などの一連の出力結果を格納したリスト：wakati_list
- 「原形」のみ格納したリスト：original_form_list

これら2つのリストを作成

In [8]:
only_hinshi_list = ['名詞', '形容詞', '動詞', '副詞']


wakati_list2 = []# 「単語、品詞、原形、」などの一連の出力結果を格納
original_form_list2 =[]# 「原形」のみ格納

# parse() の出力結果の最後は「EOS」という文字のみ
# EOSのとき、pos = wakati.split('\t')[3]の要素はないので下記forループを実行すると「list index out of range」とエラーを発生させてしまう
# よってEOSのときは条件分岐if~breakでforループから抜け出すよう記述

# wakati.split('\t')[3]の要素を見ると「名詞-固有名詞-一般」となっているので純粋な品詞名は「-」で区切ったときの0番目の要素にある
# よってpos.split('-')[0]にすることで純粋な品詞名['名詞', '形容詞', '動詞', '副詞']を指定できる
# 参考URL：https://teratail.com/questions/91987
for wakati in wakati_text:
    surface = wakati.split('\t')[0]
    if surface == 'EOS':
        break
    else:
        pos = wakati.split('\t')[3]
        if pos.split('-')[0] in only_hinshi_list:
            wakati_list2.append(wakati)
            original_form_list2.append(wakati.split('\t')[2])# 「原形」はインデックス2番目の要素にあるので

In [9]:
wakati_list2

['第一種過誤\tダイイッシュカゴ\t第一種過誤\t名詞-固有名詞-一般\t\t',
 'エラー\tエラー\tエラー\t名詞-サ変接続\t\t',
 'いう\tイウ\tいう\t動詞-自立\t五段・ワ行促音便\t基本形',
 'の\tノ\tの\t名詞-非自立-一般\t\t',
 'あわ\tアワ\tあわ\t名詞-一般\t\t',
 'てんぼう\tテンボウ\tてんぼう\t名詞-サ変接続\t\t',
 'かけ\tカケ\tかける\t動詞-自立\t一段\t連用形',
 'あわ\tアワ\tあわ\t名詞-固有名詞-地域-一般\t\t',
 'てんぼう\tテンボウ\tてんぼう\t名詞-サ変接続\t\t',
 '本当は\tホントウハ\t本当は\t副詞-一般\t\t',
 'ない\tナイ\tない\t形容詞-自立\t形容詞・アウオ段\t基本形',
 '2\t2\t2\t名詞-数\t\t',
 '集団\tシュウダン\t集団\t名詞-一般\t\t',
 '差異\tサイ\t差異\t名詞-一般\t\t',
 'ある\tアル\tある\t動詞-自立\t五段・ラ行\t基本形',
 'よう\tヨウ\tよう\t名詞-非自立-助動詞語幹\t\t',
 '判定\tハンテイ\t判定\t名詞-サ変接続\t\t',
 'し\tシ\tする\t動詞-自立\tサ変・スル\t連用形',
 'しまう\tシマウ\tしまう\t動詞-非自立\t五段・ワ行促音便\t基本形',
 '論理\tロンリ\t論理\t名詞-一般\t\t',
 '第二種\tダイニシュ\t第二種\t名詞-固有名詞-一般\t\t',
 '過誤\tカゴ\t過誤\t名詞-一般\t\t',
 'エラー\tエラー\tエラー\t名詞-サ変接続\t\t',
 'いい\tイイ\tいう\t動詞-自立\t五段・ワ行促音便\t連用形',
 'ベーター\tベーター\tベーター\t名詞-固有名詞-一般\t\t',
 'ぼんやり\tボンヤリ\tぼんやり\t副詞-一般\t\t',
 'かけ\tカケ\tかける\t動詞-自立\t一段\t連用形',
 'ぼんやり\tボンヤリ\tぼんやり\t名詞-サ変接続\t\t',
 'し\tシ\tする\t動詞-自立\tサ変・スル\t連用形',
 'い\tイ\tいる\t動詞-非自立\t一段\t連用形',
 'こちら\tコチラ\tこちら\t名詞-代名詞-一般\t\t',

In [10]:
original_form_list2

['第一種過誤',
 'エラー',
 'いう',
 'の',
 'あわ',
 'てんぼう',
 'かける',
 'あわ',
 'てんぼう',
 '本当は',
 'ない',
 '2',
 '集団',
 '差異',
 'ある',
 'よう',
 '判定',
 'する',
 'しまう',
 '論理',
 '第二種',
 '過誤',
 'エラー',
 'いう',
 'ベーター',
 'ぼんやり',
 'かける',
 'ぼんやり',
 'する',
 'いる',
 'こちら',
 'ぼんやり',
 'する',
 'いる',
 '本当は',
 'ある',
 'はず',
 '貴重',
 '2',
 '集団',
 '差異',
 '見逃す',
 'しまう',
 '様',
 '表現',
 'する',
 'いる']